# Eat Safe, Love

## Notebook Set Up

In [1]:
# Dependencies
from pymongo import MongoClient
from bson import ObjectId 
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
collections = db.list_collection_names()
collections

['establishments']

In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question:
    * Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# review a document in the establishments collection
doc_one = establishments.find_one()
pprint(doc_one)

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66b0e7ab5532ef9f2ff2dc40'),
 'geocode': {'latitude': 51.086058, 'longitude': 1.196408},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extrac

In [7]:
# Find the establishments with a hygiene score of 20
query = {
        'scores.Hygiene': 20
}

results = list(establishments.find(query))

# Display the first document in the results using pprint
pprint(results[0])

{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66b0e7ac5532ef9f2ff2f771'),
 'geocode': {'latitude': 50.769705, 'longitude': 0.27694},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate

In [8]:
# Use count_documents to display the number of documents in the result
cnt_results = establishments.count_documents(query)
print(f'There are {cnt_results} establishments have a hygiene score equal to 20.')

There are 41 establishments have a hygiene score equal to 20.


In [9]:
# Define a flattten function to handle the json file
def flatten_records(record):
    # Create a copy of original data, so any modification wouldn't affect the original data
    flat_records = record.copy()
    
    # Handle the geocode fields
    if 'geocode' in record:
        flat_records['latitude'] = record['geocode'].get('latitude')
        flat_records['longitude'] = record['geocode'].get('longitude')
        del flat_records['geocode']      
        
    # Handle the scores fields
    if 'scores' in record:
        for key, value in flat_records['scores'].items():
            flat_records[f'{key}_scores'] = value
        del flat_records['scores']
    
    # Handle the meta fields
    if 'meta' in record:
        for key, value in flat_records['meta'].items():
            flat_records[f'{key}_meta'] = value
        del flat_records['meta']
    
    # Hanlde the links fields
    # Check the links fields exists and at least contains one element.
    if 'links' in record and isinstance(record['links'], list) and len(record['links']) > 0:
        flat_records['links_ref'] = record['links'][0].get('href')  # Only keep the website links
        del flat_records['links']
    
    return flat_records

# Processing the json file
processed_records = [flatten_records(record) for record in results]

# Convert the result to a Pandas DataFrame
hygiene_df = pd.DataFrame(processed_records)

#review the DataFrame
hygiene_df

,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,ConfidenceInManagement_scores,dataSource_meta,extractDate_meta,itemCount_meta,returncode_meta,totalCount_meta,totalPages_meta,pageSize_meta,pageNumber_meta,links_ref
0,66b0e7ac5532ef9f2ff2f771,110681,0,4029,The Chase Rest Home,Caring Premises,5,5-6 Southfields Road,Eastbourne,East Sussex,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
1,66b0e7ac5532ef9f2ff2faf7,612039,0,1970/FOOD,Brenalwood,Caring Premises,5,Hall Lane,Walton-on-the-Naze,Essex,...,30,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
2,66b0e7ac5532ef9f2ff2fdf3,730933,0,1698/FOOD,Melrose Hotel,Hotel/bed & breakfast/guest house,7842,53 Marine Parade East,Clacton On Sea,Essex,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
3,66b0e7ac5532ef9f2ff2ffef,172735,0,PI/000023858,Seaford Pizza,Takeaway/sandwich shop,7844,4 High Street,Seaford,East Sussex,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
4,66b0e7ac5532ef9f2ff2fffb,172953,0,PI/000024532,Golden Palace,Restaurant/Cafe/Canteen,1,5 South Street,Seaford,East Sussex,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
5,66b0e7ac5532ef9f2ff3099d,512854,0,12/00816/BUTH,Ashby's Butchers,Retailers - other,4613,777 Southchurch Road,Southend-On-Sea,Essex,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
6,66b0e7ac5532ef9f2ff30bcf,1537089,0,22/00224/RESTUN,South Sea Express Cuisine,Restaurant/Cafe/Canteen,1,33 Alexandra Street,Southend-on-sea,Essex,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
7,66b0e7ad5532ef9f2ff320ec,155648,0,EH/00006058,Golden Palace,Takeaway/sandwich shop,7844,7 London Road,Rayleigh,Essex,...,30,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
8,66b0e7ad5532ef9f2ff3254b,1012883,0,17/00110/MIXED/S,The Tulip Tree,Restaurant/Cafe/Canteen,1,3 The Village,Chiddingstone,KENT,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
9,66b0e7ad5532ef9f2ff32d41,644109,0,41811,F & S,Retailers - other,4613,,81 Southernhay,Basildon,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...


In [10]:
# Display the number of rows in the DataFrame
num_hygiene_df = hygiene_df.shape[0]
print(f'Hygiene records has {num_hygiene_df} rows.')

Hygiene records has 41 rows.


In [11]:
# Display the first 10 rows of the DataFrame
hygiene_df.head(10)

,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,ConfidenceInManagement_scores,dataSource_meta,extractDate_meta,itemCount_meta,returncode_meta,totalCount_meta,totalPages_meta,pageSize_meta,pageNumber_meta,links_ref
0,66b0e7ac5532ef9f2ff2f771,110681,0,4029,The Chase Rest Home,Caring Premises,5,5-6 Southfields Road,Eastbourne,East Sussex,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
1,66b0e7ac5532ef9f2ff2faf7,612039,0,1970/FOOD,Brenalwood,Caring Premises,5,Hall Lane,Walton-on-the-Naze,Essex,...,30,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
2,66b0e7ac5532ef9f2ff2fdf3,730933,0,1698/FOOD,Melrose Hotel,Hotel/bed & breakfast/guest house,7842,53 Marine Parade East,Clacton On Sea,Essex,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
3,66b0e7ac5532ef9f2ff2ffef,172735,0,PI/000023858,Seaford Pizza,Takeaway/sandwich shop,7844,4 High Street,Seaford,East Sussex,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
4,66b0e7ac5532ef9f2ff2fffb,172953,0,PI/000024532,Golden Palace,Restaurant/Cafe/Canteen,1,5 South Street,Seaford,East Sussex,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
5,66b0e7ac5532ef9f2ff3099d,512854,0,12/00816/BUTH,Ashby's Butchers,Retailers - other,4613,777 Southchurch Road,Southend-On-Sea,Essex,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
6,66b0e7ac5532ef9f2ff30bcf,1537089,0,22/00224/RESTUN,South Sea Express Cuisine,Restaurant/Cafe/Canteen,1,33 Alexandra Street,Southend-on-sea,Essex,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
7,66b0e7ad5532ef9f2ff320ec,155648,0,EH/00006058,Golden Palace,Takeaway/sandwich shop,7844,7 London Road,Rayleigh,Essex,...,30,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
8,66b0e7ad5532ef9f2ff3254b,1012883,0,17/00110/MIXED/S,The Tulip Tree,Restaurant/Cafe/Canteen,1,3 The Village,Chiddingstone,KENT,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
9,66b0e7ad5532ef9f2ff32d41,644109,0,41811,F & S,Retailers - other,4613,,81 Southernhay,Basildon,...,20,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...


In [13]:
# Save the data into a csv file
output_file_path = './Outputs/hygiene_records.csv'
hygiene_df.to_csv(output_file_path)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [17]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.

query = {
    'RatingValue' : {'$gte': 4},
    'LocalAuthorityName': {
        '$regex': 'London',
        '$options': 'i' # The specified matching pattern for the regular expression, 'i' indicates case-insensitive matching.
    }
}

rating_value_results = list(establishments.find(query))

# Display the first document in the results using pprint
pprint(rating_value_results[0])

{'AddressLine1': 'Oak Apple Farm Building 103 Sheernes Docks',
 'AddressLine2': 'Sheppy Kent',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': "Charlie's",
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4627.439467780196,
 'FHRSID': 621707,
 'LocalAuthorityBusinessID': 'PI/000025307',
 'LocalAuthorityCode': '508',
 'LocalAuthorityEmailAddress': 'publicprotection@cityoflondon.gov.uk',
 'LocalAuthorityName': 'City of London Corporation',
 'LocalAuthorityWebSite': 'http://www.cityoflondon.gov.uk/Corporation/homepage.htm',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'ME12',
 'RatingDate': '2021-10-18T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66b0e7ac5532ef9f2ff3118c'),
 'geocode': {'latitude': 51.369321, 'longitude': 0.508551},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/621707',
            'rel': 'self'}]

In [18]:
# Use count_documents to display the number of documents in the result
cnt_rating_results = establishments.count_documents(query)
print(f'There are {cnt_rating_results} ests have ratingvalue greater than or equal 4.')

There are 33 records have ratingvalue greater than or equal 4.


In [19]:
# Using the same method to processing the json file
rating_processing_results = [flatten_records(record) for record in rating_value_results]

# Convert the result to a Pandas DataFrame
rating_df = pd.DataFrame(rating_processing_results)
rating_df

,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,ConfidenceInManagement_scores,dataSource_meta,extractDate_meta,itemCount_meta,returncode_meta,totalCount_meta,totalPages_meta,pageSize_meta,pageNumber_meta,links_ref
0,66b0e7ac5532ef9f2ff3118c,621707,0,PI/000025307,Charlie's,Other catering premises,7841,Oak Apple Farm Building 103 Sheernes Docks,Sheppy Kent,,...,5,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
1,66b0e7ac5532ef9f2ff314b8,1130836,0,PI/000034075,Mv City Cruises Erasmus,Other catering premises,7841,Cherry Garden Pier,Cherry Garden Street Rotherhithe,Charlton,...,0,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
2,66b0e7ad5532ef9f2ff32001,293783,0,PI/000002614,Benfleet Motor Yacht Club,Other catering premises,7841,Ferry Road,South Benfleet Essex,,...,10,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
3,66b0e7ad5532ef9f2ff32e02,1315095,0,PI/000036464,Coombs Catering t/a The Lock and Key,Restaurant/Cafe/Canteen,1,Leslie Ford House,Tilbury,Charlton,...,0,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
4,66b0e7ad5532ef9f2ff32e03,294474,0,PI/000014647,Tilbury Seafarers Centre,Restaurant/Cafe/Canteen,1,Tenants Row Tilbury Docks,Tilbury Essex,,...,0,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
5,66b0e7ad5532ef9f2ff3350d,294900,0,PI/000019066,Mv Valulla,Other catering premises,7841,Reeds River Cruises LtdKings ReachRiver Thames...,London,,...,0,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
6,66b0e7ad5532ef9f2ff35614,293756,0,PI/000002538,Tereza Joanne,Other catering premises,7841,Funcraft UK Ltd King George V Dock Woolwich Ma...,London,,...,5,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
7,66b0e7ad5532ef9f2ff359e2,878523,0,PI/000029844,Brick Lane Brews,Restaurant/Cafe/Canteen,1,Air side London City Airport,London,,...,5,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
8,66b0e7ad5532ef9f2ff359f9,294606,0,PI/000016185,WH Smith,Retailers - other,4613,London City Airport,Silvertown London,,...,0,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
9,66b0e7ad5532ef9f2ff359fa,293772,0,PI/000002585,The Nuance Group (UK) Limited,Retailers - other,4613,Duty Free Shop Passenger Terminal,London City AirportRoyal DocksLondon,,...,0,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...


In [20]:
# Using shape to calculate the number of records
num_rating_df = rating_df.shape[0]

# Display the number of rows in the DataFrame
print(f'There are {num_rating_df} establishments in London have rating value greater than or equal 4.')

There are 33 establishments in London have rating value greater than or equal 4.


In [21]:
# Display the first 10 rows of the DataFrame
rating_df.head(10)

,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,ConfidenceInManagement_scores,dataSource_meta,extractDate_meta,itemCount_meta,returncode_meta,totalCount_meta,totalPages_meta,pageSize_meta,pageNumber_meta,links_ref
0,66b0e7ac5532ef9f2ff3118c,621707,0,PI/000025307,Charlie's,Other catering premises,7841,Oak Apple Farm Building 103 Sheernes Docks,Sheppy Kent,,...,5,None,0001-01-01T00:00:00,0,None,0,0,0,0,https://api.ratings.food.gov.uk/establishments...
1,66b0e7ac5532ef9f2ff314b8,1130836,0,PI/000034075,Mv City Cruises Erasmus,Other catering premises,7841,Cherry Garden Pier,Cherry Garden Street Rotherhithe,Charlton,...,0,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
2,66b0e7ad5532ef9f2ff32001,293783,0,PI/000002614,Benfleet Motor Yacht Club,Other catering premises,7841,Ferry Road,South Benfleet Essex,,...,10,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
3,66b0e7ad5532ef9f2ff32e02,1315095,0,PI/000036464,Coombs Catering t/a The Lock and Key,Restaurant/Cafe/Canteen,1,Leslie Ford House,Tilbury,Charlton,...,0,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
4,66b0e7ad5532ef9f2ff32e03,294474,0,PI/000014647,Tilbury Seafarers Centre,Restaurant/Cafe/Canteen,1,Tenants Row Tilbury Docks,Tilbury Essex,,...,0,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
5,66b0e7ad5532ef9f2ff3350d,294900,0,PI/000019066,Mv Valulla,Other catering premises,7841,Reeds River Cruises LtdKings ReachRiver Thames...,London,,...,0,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
6,66b0e7ad5532ef9f2ff35614,293756,0,PI/000002538,Tereza Joanne,Other catering premises,7841,Funcraft UK Ltd King George V Dock Woolwich Ma...,London,,...,5,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
7,66b0e7ad5532ef9f2ff359e2,878523,0,PI/000029844,Brick Lane Brews,Restaurant/Cafe/Canteen,1,Air side London City Airport,London,,...,5,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
8,66b0e7ad5532ef9f2ff359f9,294606,0,PI/000016185,WH Smith,Retailers - other,4613,London City Airport,Silvertown London,,...,0,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
9,66b0e7ad5532ef9f2ff359fa,293772,0,PI/000002585,The Nuance Group (UK) Limited,Retailers - other,4613,Duty Free Shop Passenger Terminal,London City AirportRoyal DocksLondon,,...,0,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...


In [22]:
# Save the DataFrame into a csv file.
output_file_path = './Outputs/rating_records.csv'
rating_df.to_csv(output_file_path)

3. What are the top 5 establishments with a RatingValue rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [23]:
# Query the Penang Flavours's data, only return restaurant name, latitude, longtitude.
query = {
    'BusinessName': 'Penang Flavours'
}

fileds = {
    '_id':1,
    'BusinessName':1,
    'geocode.latitude':1,
    'geocode.longitude':1
}

penang_data = list(establishments.find(query, fileds))
pprint(penang_data)

[{'BusinessName': 'Penang Flavours',
  '_id': ObjectId('66b0e9ddbd725de407565cdc'),
  'geocode': {'latitude': 51.490142, 'longitude': 0.08384}}]


In [24]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score 

degree_search = 0.01
latitude = float(penang_data[0]['geocode']['latitude'])
longitude = float(penang_data[0]['geocode']['longitude']) 
# print(latitude, longitude) 

# Search criteria: Latitude and longitude within a range of 0.01 degrees.
lat_range  = [latitude - degree_search, latitude + degree_search]
lon_range = [longitude - degree_search, longitude + degree_search]
# print(lat_range)
# print(lon_range)
match_query = {
    '$match': {
        'geocode.latitude': {'$gte': lat_range[0], '$lte': lat_range[1]},
        'geocode.longitude': {'$gte': lon_range[0], '$lte': lon_range[1]},
        'RatingValue': 5
    }
}



sort_values = {
    '$sort': {
        'scores.Hygiene': 1
    }
}

limit_values = {
    '$limit': 5
}

# Put the pipeline together
pipeline = [match_query, sort_values, limit_values]

# Run the pipeline, cast the results as a list
top_restaurants_results = list(establishments.aggregate(pipeline))

# Print the results
top_restaurants_results

[{'_id': ObjectId('66b0e7ad5532ef9f2ff34fb8'),
  'FHRSID': 1380578,
  'ChangesByServerID': 0,
  'LocalAuthorityBusinessID': '14425',
  'BusinessName': 'Howe and Co Fish and Chips - Van 17',
  'BusinessType': 'Mobile caterer',
  'BusinessTypeID': 7846,
  'AddressLine1': 'Restaurant And Premises 107A Plumstead High Street',
  'AddressLine2': '',
  'AddressLine3': 'Plumstead',
  'AddressLine4': 'Greenwich',
  'PostCode': 'SE18 1SE',
  'Phone': '',
  'RatingValue': 5,
  'RatingKey': 'fhrs_5_en-gb',
  'RatingDate': '2021-11-11T00:00:00',
  'LocalAuthorityCode': '511',
  'LocalAuthorityName': 'Greenwich',
  'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
  'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
  'scores': {'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 0},
  'SchemeType': 'FHRS',
  'geocode': {'longitude': 0.0925370007753372, 'latitude': 51.4875335693359},
  'RightToReply': '',
  'Distance': 4646.95593107927,
  'NewRatingPending': False,
  'meta

In [25]:
# Using the same method to processing the json file
top5_processing_results = [flatten_records(record) for record in top_restaurants_results]

# Convert result to Pandas DataFrame
top5_restaurants_df = pd.DataFrame(top5_processing_results)
top5_restaurants_df

,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,ConfidenceInManagement_scores,dataSource_meta,extractDate_meta,itemCount_meta,returncode_meta,totalCount_meta,totalPages_meta,pageSize_meta,pageNumber_meta,links_ref
0,66b0e7ad5532ef9f2ff34fb8,1380578,0,14425,Howe and Co Fish and Chips - Van 17,Mobile caterer,7846,Restaurant And Premises 107A Plumstead High St...,,Plumstead,...,0,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
1,66b0e7ad5532ef9f2ff34fec,694478,0,PI/000086506,Atlantic Fish Bar,Takeaway/sandwich shop,7844,35 Lakedale Road,,Plumstead,...,5,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
2,66b0e7ad5532ef9f2ff34fe6,695241,0,PI/000179088,Plumstead Manor Nursery,Caring Premises,5,Plumstead Manor School Old Mill Road,,Plumstead,...,5,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
3,66b0e7ad5532ef9f2ff34fac,695223,0,PI/000178842,Iceland,Retailers - supermarkets/hypermarkets,7840,144 - 146 Plumstead High Street,,Plumstead,...,5,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...
4,66b0e7ad5532ef9f2ff34fdc,694609,0,PI/000116619,Volunteer,Pub/bar/nightclub,7843,130 - 132 Plumstead High Street,,Plumstead,...,0,None,0001-01-01T00:00:00,0,None,0,0,0,0,http://api.ratings.food.gov.uk/establishments/...


In [26]:
# Save the results into csv file
top5_restaurants_df.to_csv('./Outputs/top5_restaurants_records.csv')

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [27]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
match_query = {
    '$match': {
        'scores.Hygiene': 0
    }
}

# 2. Groups the matches by Local Authority
group_query = {
    '$group': {'_id':{'LocalAuthorityName':'$LocalAuthorityName'},
               'count':{'$sum': 1}
               }
}

# 3. Sorts the matches from highest to lowest
sort_values = {
    '$sort': {'count': -1}
}

# Put the pipeline together
pipeline = [match_query, group_query, sort_values]

# Run the pipeline
restaurant_by_location_results = list(establishments.aggregate(pipeline))

#print
print(f'There are {len(restaurant_by_location_results)} rows in results.')

There are 55 rows in results.


In [28]:
restaurant_by_location_results[0:10]

[{'_id': {'LocalAuthorityName': 'Thanet'}, 'count': 1130},
 {'_id': {'LocalAuthorityName': 'Greenwich'}, 'count': 882},
 {'_id': {'LocalAuthorityName': 'Maidstone'}, 'count': 713},
 {'_id': {'LocalAuthorityName': 'Newham'}, 'count': 711},
 {'_id': {'LocalAuthorityName': 'Swale'}, 'count': 686},
 {'_id': {'LocalAuthorityName': 'Chelmsford'}, 'count': 680},
 {'_id': {'LocalAuthorityName': 'Medway'}, 'count': 672},
 {'_id': {'LocalAuthorityName': 'Bexley'}, 'count': 607},
 {'_id': {'LocalAuthorityName': 'Southend-On-Sea'}, 'count': 586},
 {'_id': {'LocalAuthorityName': 'Tendring'}, 'count': 542}]

In [29]:
# Flatten the data
flatten_data = [
    {
        'LocalAuthorityName' : item['_id']['LocalAuthorityName'],
        'count': item['count']
    } for item in restaurant_by_location_results
]

# Convert the result to a Pandas DataFrame
restaurant_by_location_df = pd.DataFrame(flatten_data)
restaurant_by_location_df

,LocalAuthorityName,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542


In [30]:
# Display the number of rows in the DataFrame
num_restaurant_by_location_df = restaurant_by_location_df.shape[0]
print(f'There are {num_restaurant_by_location_df} rows in results.')

There are 55 rows in results.


In [31]:
# Display the first 10 rows of the DataFrame
restaurant_by_location_df.head(10)

,LocalAuthorityName,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542


In [32]:
# Save the results
restaurant_by_location_df.to_csv('./Outputs/restaurant_by_location_records.csv')